## [First Round](https://resultadoshistorico.onpe.gob.pe/EG2021/ResumenGeneral/10/T)

In [104]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
import pandas as pd
import numpy as np
import os
import time
import re
import unidecode
import time 
from selenium.webdriver.support.ui import Select

# Driver Path Address

In [16]:
driver_path = r"..\chromedriver.exe"
driver = webdriver.Chrome( driver_path )
# Maximize window
driver.maximize_window()

### Extracting all tables

In [100]:
driver.get( 'https://resultadoshistorico.onpe.gob.pe/EG2021/ResumenGeneral/10/T' )

## Pesidential results

In [101]:
# presidential
presidential = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-menu/div/nav/div/div/div[2]/div/div[2]/a" )
presidential.click()### Get all elements from scope

scope = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select" )

# All selenium objects in scope
scope_options = [ x for x in scope.find_elements_by_tag_name("option") ]
dict_scope_options = { option.text : option for option in scope_options }

dict_scope_options['PERÚ'].click()

### Get all elements from scope

In [74]:
scope = driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div/select" )

# All selenium objects in scope
scope_options = [ x for x in scope.find_elements_by_tag_name("option") ]
dict_scope_options = { option.text : option for option in scope_options }

In [ ]:
dict_scope_options['PERÚ'].click()

### Loop over all departments

In [105]:
# Store all_tables
all_tables = {}
# All selenium objects in department select
dpt = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )
# Get number of total options
num_prov_options = len( dpt.options )

for dpt_idx in range( num_prov_options ):
    
    # Get again all departments since HTML is refreshing
    # all elements
    
    dpt = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[2]/select" ) )
    department = dpt.options[ dpt_idx ]
    
    # Get departmant name
    dpt_name = department.text
    
    if dep_name != "--TODOS--" :
        
        # click on department
        department.click()
        
        # Get all elements of province
        prov = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
        num_prov_options = len( prov.options )
        
        for prov_idx in range( num_prov_options ):
            
            # Get again all districts since HTML is refreshing
            # all elements
            prov = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[3]/select" ) )
            province = prov.options[ prov_idx ]
                
            # Get province name
            prov_name = province.text
            
            if prov_name != "--TODOS--" :
                
                # click on province
                province.click()
                
                # Get all elements from district
                dist = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                num_dist_options = len( dist.options )
                
                for dist_idx in range( num_dist_options ):
                    
                    # Get again all districts since HTML is refreshing
                    # all elements
                    dist = Select( driver.find_element_by_xpath( "/html/body/onpe-root/onpe-layout-container/onpe-onpe-epres-re/div[1]/div[3]/div[1]/div[1]/div/div/div[4]/select" ) )
                    district = dist.options[ dist_idx ]
                    
                    # Get district name
                    dist_name = district.text
                    
                    if dist_name != "-- SELECCIONE --" :
                        
                        # click on district
                        district.click()
                        
                        # Get UBIGEO
                        ubigeo = driver.current_url.split("/")[ -1 ]
                        
                        ## Get table of presidential votes
                        # Get html at this point
                        html = driver.page_source
                        soup = BeautifulSoup( html, 'html.parser' )

                        # Read the table using pandas
                        table_selector = soup.select_one( "#table1" )
                        table = pd.read_html( str( table_selector ) )
                        
                        # Cleaning tables
                        row_new_columns = table[ 0 ].iloc[ 0 , 2: ]
                        clean_columns = row_new_columns \
                                              .str.replace( " ", "_") \
                                              .str.lower().str.replace( "%", "share_") \
                                              .apply( lambda x : unidecode.unidecode( x ) ) \
                                              .tolist()
                        
                        # Selecting specific columns
                        table_clean = table[0].iloc[ 1:, 2: ].copy()
                        
                        # rename columns
                        table_clean.columns = clean_columns
                        
                        # New values to columns 
                        table_clean[ 'department' ] = dpt_name
                        table_clean[ 'province' ]   = prov_name
                        table_clean[ 'district' ]   = dist_name
                        table_clean[ 'ubigeo' ]     = ubigeo
                        
                        # store tables
                        all_tables[ ubigeo ] = table_clean

In [109]:
final_data = pd.concat( all_tables.values() ).reset_index( drop = True )

In [111]:
final_data.to_excel( r'../A_MicroData/pd_elections_2021_first_round.xlsx' , index = False )